<a href="https://colab.research.google.com/github/haiderds/EdvancerIITK-Quora/blob/main/first_iteration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import tensorflow as tf
import tensorflow_hub as hub
#tf.compat.v1.disable_eager_execution()
#from sentence_transformers import SentenceTransformer
from sentence_transformers import SentenceTransformer
import time
from keras.models import Sequential
from keras.layers import Dense
#create model
model = Sequential()

In [ ]:
#common functions
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix')

    print(cm)

In [ ]:
# load the data
full_df = pd.read_csv('train.csv')

FileNotFoundError: ignored

In [ ]:
full_df.head(10)

In [ ]:
full_df.target.unique()

In [ ]:
#there is big class imbalance so lets apply class balancing technique
full_df['target'].value_counts().plot.bar()

In [ ]:
class_count_0, class_count_1 = full_df['target'].value_counts()

# Separate class
class_0 = full_df[full_df['target'] == 0]
class_1 = full_df[full_df['target'] == 1]
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

In [ ]:
#code for under sampling
class_0_under = class_0.sample(class_count_1)

test_under = pd.concat([class_0_under, class_1], axis=0)

print("total class of 1 and0:",test_under['target'].value_counts())# plot the count after under-sampeling
test_under['target'].value_counts().plot(kind='bar', title='count (target)')

In [ ]:
# #plot the count after over-sampling

# class_1_over = class_1.sample(class_count_0, replace=True)

# test_over = pd.concat([class_1_over, class_0], axis=0)

# print("total class of 1 and 0:",test_over['target'].value_counts())
# test_over['target'].value_counts().plot(kind='bar', title='count (target)')

In [ ]:
# split test train with 70 percentage training data and 30 percent test data

X_train, X_test, y_train, y_test = train_test_split(test_under['question_text'], test_under['target'],
                                   random_state=104, 
                                   test_size=0.25, 
                                   shuffle=True)

print('Stats for Target for Training Data \n {}'.format(y_train.value_counts()))
print('Stats for Target for Test Data \n {}'.format(y_test.value_counts()))

In [ ]:
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
#sample_count = 60000

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
start_time = time.time()
sentence_embeddings = sbert_model.encode(X_train.to_list())
#X_train.to_list()
#sbert_model.encode(['This is a test','only for testing purpose','Dog is an animal'])
print(time.time() - start_time)

In [ ]:
sentence_embeddings

In [ ]:
from tensorflow.keras.optimizers import SGD
#create model

model = Sequential()


#add model layers
model.add(Dense(400, activation='relu', input_shape=(768,)))
model.add(Dense(400, activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
opt = SGD(lr=0.01, momentum=0.9)
model.compile(optimizer=opt, loss='binary_crossentropy',metrics=['categorical_accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping
#set early stopping monitor so the model stops training when it won't improve anymore
early_stopping_monitor = EarlyStopping(patience=3)
#train model
model.fit(sentence_embeddings, y_train, epochs=50, batch_size=20)

In [ ]:
y_train.unique()

In [ ]:
start_time = time.time()
sentence_embeddings_test = sbert_model.encode(X_test.to_list())
#X_train.to_list()
#sbert_model.encode(['This is a test','only for testing purpose','Dog is an animal'])
print(time.time() - start_time)

In [ ]:
from sklearn.metrics import confusion_matrix,roc_auc_score,f1_score
y_pred=model.predict(sentence_embeddings_test)

cnf_matrix=confusion_matrix(y_test,y_pred.round())
plot_confusion_matrix(cnf_matrix,classes=[0,1])
plt.show()

In [ ]:
score = roc_auc_score(y_test, y_pred)
print(f"ROC AUC: {score:.4f}")

In [ ]:
f1_score(y_test, y_pred.round())